In [97]:
# %%writefile tablereader_js2.py
from pymongo import MongoClient
import pprint 
import pandas as pd 
import copy
from bs4 import BeautifulSoup as bs
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

import warnings

warnings.filterwarnings('ignore')




# send GET request using selenium and check status
option = webdriver.ChromeOptions()
option.add_argument(' - incognito')

browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver', chrome_options=option)
site_url = 'https://www.congress.gov/search?q=%7B%22source%22%3A%22legislation%22%7D'

req = requests.get(site_url)
print('_______________')
print('_______________')
print('Request Status Code: {}'.format(req.status_code))

browser.get(site_url)

# # save html into mongo
# client = MongoClient('mongodb://localhost:27017/')
# db = client.bills
# pages = db.pages
# pages.insert_one({'html': req.content})

# parse the html with BeautifulSoup
soup = bs(browser.page_source, 'lxml')
print('--------------')
print('--------------')
print(soup.title)
print('--------------')
# print(soup.prettify())

# table of bills are in ol class
# navigate to find bill info
div = soup.find('div', {'class':'search-column-main'})
table = div.find('ol')
# print(tab.prettify())
# print(len(table))




# #  need to iterate though each li class to find most of the above
rows = table.find_all('li', {'class':'expanded'})


# print(len(rows))
print(len(rows[0]))
pprint.pprint(rows[0])

_______________
_______________
Request Status Code: 200
--------------
--------------
<title>Legislative Search Results | Congress.gov | Library of Congress</title>
--------------
15
<li class="expanded" style="display: block;"> <div><span class="visualIndicator">BILL</span></div>
    1.
    <span class="result-heading"><a href="https://www.congress.gov/bill/115th-congress/house-bill/7211?r=1">H.R.7211</a> — 115th Congress (2017-2018)</span>
<span class="result-title">To amend the Mineral Leasing Act to authorize the Secretary of the Interior to regulate hydraulic fracturing operations on Federal lands, and for other purposes.</span>
<span class="result-item">
<strong>Sponsor:</strong> <a href="/member/darren-soto/S001200" target="_blank">Rep. Soto, Darren [D-FL-9]</a> (Introduced 11/30/2018) <strong>Cosponsors:</strong> (<a href="https://www.congress.gov/bill/115th-congress/house-bill/7211/cosponsors?r=1&amp;overview=closed#tabs">0</a>)        </span>
<span class="result-item">
<stro

In [162]:
# store each row in a dictionary
empty_row = {'bill_id': None, 
            'bill_url': None, 
            'congress_id': None,
            'desc': None,
            'sponsor': None, 
#             'cosponsors': None,  #requires navigation to another url and extracting names from table
            'committee': None, 
            'bill_status': None,
            'body': None
            }


In [100]:
a = rows[0].find_all('a')
# print(len(a))
pprint.pprint(a)

bill_id = a[0].text
bill_url = a[0]['href']
sponsor = a[1].text

[<a href="https://www.congress.gov/bill/115th-congress/house-bill/7211?r=1">H.R.7211</a>,
 <a href="/member/darren-soto/S001200" target="_blank">Rep. Soto, Darren [D-FL-9]</a>,
 <a href="https://www.congress.gov/bill/115th-congress/house-bill/7211/cosponsors?r=1&amp;overview=closed#tabs">0</a>,
 <a href="https://www.congress.gov/bill/115th-congress/house-bill/7211/all-actions?r=1&amp;overview=closed#tabs">All Actions</a>]


In [149]:
a[0]['href'].strip()

'https://www.congress.gov/bill/115th-congress/house-bill/7211?r=1'

In [109]:
s = rows[0].find_all('span')
print(len(s))
pprint.pprint(s)

7
[<span class="visualIndicator">BILL</span>,
 <span class="result-heading"><a href="https://www.congress.gov/bill/115th-congress/house-bill/7211?r=1">H.R.7211</a> — 115th Congress (2017-2018)</span>,
 <span class="result-title">To amend the Mineral Leasing Act to authorize the Secretary of the Interior to regulate hydraulic fracturing operations on Federal lands, and for other purposes.</span>,
 <span class="result-item">
<strong>Sponsor:</strong> <a href="/member/darren-soto/S001200" target="_blank">Rep. Soto, Darren [D-FL-9]</a> (Introduced 11/30/2018) <strong>Cosponsors:</strong> (<a href="https://www.congress.gov/bill/115th-congress/house-bill/7211/cosponsors?r=1&amp;overview=closed#tabs">0</a>)        </span>,
 <span class="result-item">
<strong>Committees:</strong> House - Natural Resources, Energy and Commerce        </span>,
 <span class="result-item"><strong> Latest Action:            </strong> House - 11/30/2018 Referred to the Committee on Natural Resources, and in addition

In [155]:
pclass = rows[0].find_all('p')
print(len(pclass))
pprint.pprint(pclass)

2
[<p class="hide_fromsighted">This bill has the status Introduced</p>,
 <p class="hide_fromsighted">Here are the steps for Status of Legislation:</p>]


In [160]:
pclass[0].text[25:]

'Introduced'

In [165]:
all_rows = []

for row in rows:
    new_row = copy.copy(empty_row)

    columns = row.find_all('a')
    new_row['bill_id'] = columns[0].text.strip()
    new_row['bill_url'] = columns[0]['href'].strip()
    new_row['sponsor'] = columns[1].text.strip()
    
    columns = row.find_all('span')
    new_row['congress_id'] = columns[1].text
    new_row['desc'] = columns[2].text
    new_row['committee'] = columns[4].text.strip()[12:]
    
    columns = row.find_all('p')
    new_row['bill_status'] = columns[0].text.strip()[25:]
    
    all_rows.append(new_row)

In [171]:
all_rows[-5:]

[{'bill_id': 'H.R.7116',
  'bill_url': 'https://www.congress.gov/bill/115th-congress/house-bill/7116?r=96',
  'congress_id': 'H.R.7116 — 115th Congress (2017-2018)',
  'desc': 'Suicide Prevention Assistance Act',
  'sponsor': 'Rep. DeSaulnier, Mark [D-CA-11]',
  'committee': 'House - Energy and Commerce',
  'bill_status': 'Introduced',
  'body': None},
 {'bill_id': 'H.R.7115',
  'bill_url': 'https://www.congress.gov/bill/115th-congress/house-bill/7115?r=97',
  'congress_id': 'H.R.7115 — 115th Congress (2017-2018)',
  'desc': '3D Firearms Prohibitions Act',
  'sponsor': 'Rep. Pallone, Frank, Jr. [D-NJ-6]',
  'committee': 'House - Energy and Commerce, Judiciary',
  'bill_status': 'Introduced',
  'body': None},
 {'bill_id': 'H.R.7114',
  'bill_url': 'https://www.congress.gov/bill/115th-congress/house-bill/7114?r=98',
  'congress_id': 'H.R.7114 — 115th Congress (2017-2018)',
  'desc': 'Natural Disaster Safe Inmate Relocation Act',
  'sponsor': 'Rep. Meng, Grace [D-NY-6]',
  'committee': 'H

In [ ]:
    # # # initialize iterator over the rows in the table
    # rows = tab.find_all('span')


    # for row in rows[1:]:
    #     new_row = copy.copy(empty_row)

    #     # a list of all he entries in the row
    #     columns = row.find_all('td')
        # print(columns)
    #     # new_row['bill_id'] = columns[0].text.strip())
    #     # new_row['bill_desc'] = columns[0].text.strip())
    #     # new_row['intro_date'] = columns[0].text.strip())
    #     # new_row['bill_url'] = columns[0].text.strip())
    #     # new_row['body'] = columns[0].text.strip())

In [ ]:
if __name__ == '__main__':
    site_table_reader()